# Create and Run chat flows using Promptflow Python SDK 

### Overview
Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality. 

In this hands-on, you will be able to:
Create flows that link fine-tuned phi3.5 endpoint(Python code) and gpt model in a executable workflow.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.


#### 1. Create Promptflow client with Credential and configuration
#### 2. Run Basic Promptflow with questions to compare models
#### 3. Run Context Added Promptflow with the outdoor questions 

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [22]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from evaluate import run_azure_flow, run_azure_eval_flow
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError

with open('./config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

3d4d3dd0-79d4-40cf-a94e-b4154812c6ca
slm-innovator-rg
slm-pjt1


In [23]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## Create Promptflow client with Credential and configuration
- Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name

In [4]:
credential = DefaultAzureCredential()
# if you cannot use DefaultAzureCredential, you need to set up the environment variables for the service principal
# credential = ServicePrincipalCredentials(
#     client_id=os.environ["AZURE_CLIENT_ID"],
#     secret=os.environ["AZURE_CLIENT_SECRET"],
#     tenant=os.environ["AZURE_TENANT_ID"],
# )
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")

pf_azure_client = PFClient.from_config(credential=credential, path="./config.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


Found the config file in: config.json


Connected to Azure AI Studio Workspace: slm-pjt1
Workspace Location: eastus
Workspace ID: /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


## Check the exist connections
- currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.

In [7]:
from promptflow.client import PFClient
from promptflow.entities import AzureOpenAIConnection, CustomConnection

# Get a pf client to get connections
# currently we only support create connection in Azure AI, ML Studio UI
pf = PFClient()
try:
    pf.connections.list()
    for connection in pf.connections.list():
        print(connection)
except (ValueError, TypeError) as e:
    print(e)

auth_mode: key
name: ai-slmhub1616907550322_aoai
module: promptflow.connections
created_date: '2024-10-21 23:44:10.828648+00:00'
last_modified_date: '2024-10-21 23:44:10.828648+00:00'
type: azure_open_ai
api_key: '******'
api_base: https://ai-slmhub1616907550322.openai.azure.com/
api_type: Azure
api_version: 2023-07-01-preview
resource_id: 
  /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.CognitiveServices/accounts/ai-slmhub1616907550322

auth_mode: key
name: ai-slmhub1616907550322
module: promptflow.connections
created_date: '2024-10-21 23:44:10.828648+00:00'
last_modified_date: '2024-10-21 23:44:10.828648+00:00'
type: azure_ai_services
api_key: '******'
endpoint: https://ai-slmhub1616907550322.cognitiveservices.azure.com/

auth_mode: key
name: wsstq75qvfcwosearch
module: promptflow.connections
created_date: '2024-10-22 08:12:14.702733+00:00'
last_modified_date: '2024-10-22 08:12:14.702733+00:00'
type: cognitive_search
api_key: 

## Create a chat flow using Promptflow client
- Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI studio UI. 

In [11]:
#Storage Blob Data Contributor, Storage File Data Privileged Contributor role을 managed Identity / service principal / InteractiveBrowserCredential 에게 부여하면 아래 코드를 실행할 수 있습니다.
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")

Flow created successfully:
{
    "name": "1b85e066-c15e-4532-a873-e78773badc15",
    "type": "chat",
    "description": "fine-tuned model comparison flow",
    "path": "Users/8e2870f5-6900-44d5-a7ca-5892562926fd/promptflow/chat-10-23-2024-13-43-47/flow.dag.yaml",
    "code": "azureml://locations/eastus/workspaces/884cbbaf-4d30-486f-a017-f9265971b9e1/flows/1b85e066-c15e-4532-a873-e78773badc15",
    "display_name": "comparison flow created from python sdk",
    "owner": {
        "user_object_id": "8e2870f5-6900-44d5-a7ca-5892562926fd",
        "user_tenant_id": "16b3c013-d300-468d-ac64-7eda0820b6d3",
        "user_name": "7ac00d14-31c3-4eac-a9a9-338f6dbd1c98"
    },
    "is_archived": false,
    "created_date": "2024-10-23 13:43:49.700823+00:00",
    "flow_portal_url": "https://ai.azure.com/projectflows/1b85e066-c15e-4532-a873-e78773badc15/884cbbaf-4d30-486f-a017-f9265971b9e1/details/Flow?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/

## Run Basic Promptflow with questions to compare models
- Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models

In [16]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


Uploading chat (0.04 MBs): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36388/36388 [00:00<00:00, 555802.79it/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_variant_0_20241023_135222_031989/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [17]:
monitor_status(pf_azure_client, run_result)

Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:00<00:01,  1.65step/s]

Current Status: Preparing
Current Status: Preparing


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:21<00:12, 12.66s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:04<00:00, 21.36s/step]

Promptflow Running Completed


In [18]:
detail = pf_azure_client.get_details(run_result)

detail

inputs.question  \
outputs.line_number                                                    
0                                       what is the center of Seoul?   
1                                     What is the capital of France?   
2                       Tell me about the brief history of Microsoft   
3                                        Who wrote Romeo and Juliet?   
4                                          What does HTML stand for?   
5                                         Who painted the Mona Lisa?   
6                    What is the largest planet in our solar system?   
7                     Who won the Nobel Prize in Literature in 2024?   

                     inputs.line_number  \
outputs.line_number                       
0                                     0   
1                                     1   
2                                     2   
3                                     3   
4                                     4   
5                                     5   
6                                     6   
7                                     7   

                                                  outputs.phi35_answer  \
outputs.line_number                                                      
0                     Gyeongbokgung Palace is the most prominent hi...   
1                                                                Paris   
2                     Microsoft was founded on April 4, 1975, by Bi...   
3                                                  William Shakespeare   
4                           HTML stands for HyperText Markup Language.   
5                             Leonardo da Vinci painted the Mona Lisa.   
6                     Jupiter is the largest planet in our solar sy...   
7                     As of my knowledge cutoff in 2023, I cannot p...   

                                                  outputs.gpt4o_answer  
outputs.line_number                                                     
0                    The center of Seoul is often considered to be ...  
1                                      The capital of France is Paris.  
2                    Microsoft was founded on April 4, 1975, by Bil...  
3                        William Shakespeare wrote "Romeo and Juliet."  
4                           HTML stands for HyperText Markup Language.  
5                      The Mona Lisa was painted by Leonardo da Vinci.  
6                    The largest planet in our solar system is Jupi...  
7                    I'm sorry, I don't have that information. You ...

## Run Context Added Promptflow with the outdoor questions 

In [25]:
#TODO test https://github.com/microsoft/promptflow/blob/main/examples/flows/chat/chat-with-pdf/chat-with-pdf-azure.ipynb
#2. Run a flow with setting (context size 2K)
flow_path = "./chatwithcontext"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_with_context_and_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


Uploading chatwithcontext (0.04 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38890/38890 [00:00<00:00, 557424.97it/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chatwithcontext_variant_0_20241023_142917_923363/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [26]:
monitor_status(pf_azure_client, run_result_with_context)

Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:00<00:01,  1.44step/s]

Current Status: Preparing
Current Status: Preparing


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:22<00:12, 12.93s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:05<00:00, 81.71s/step]

Promptflow Running Completed


In [13]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

inputs.question  \
outputs.line_number                                                      
0                                    tell me about your TrailMaster X4   
1                                       Do you have any climbing gear?   
2                       Can you tell me about your selection of tents?   
3                    Do you have TrekReady Hiking Boots? How much i...   
4                              can you tell me BaseCamp Folding Table?   
5                              What is your return or exchange policy?   
6                    I would like to return the tent I bought. It i...   

                                                        inputs.context  \
outputs.line_number                                                      
0                    TrailMaster X4 Tent is a durable polyester ten...   
1                    TrailMaster X4 Tent is a durable polyester ten...   
2                    TrailMaster X4 Tent is a durable polyester ten...   
3                    TrailMaster X4 Tent is a durable polyester ten...   
4                    TrailMaster X4 Tent is a durable polyester ten...   
5                    TrailMaster X4 Tent is a durable polyester ten...   
6                    TrailMaster X4 Tent is a durable polyester ten...   

                     inputs.line_number  \
outputs.line_number                       
0                                     0   
1                                     1   
2                                     2   
3                                     3   
4                                     4   
5                                     5   
6                                     6   

                                                  outputs.phi35_answer  \
outputs.line_number                                                      
0                     🌄 **TrailMaster X4 행복한 산길 여행의 정수!**\n\nTrailM...   
1                                                                 None   
2                     🌟 테크노 피트와 탐험 소지에 열정을 가진 당신에게, 우리의 옵션 세트에서 테크노...   
3                                                                 None   
4                                                                 None   
5                                                                 None   
6                                                                 None   

                                                  outputs.gpt4o_answer  
outputs.line_number                                                     
0                    TrailMaster X4에 대해 알려드릴게요!\n\nTrailMaster X4는 ...  
1                    아쉽지만, 저는 실제 장비를 가지고 있진 않아요. 😅 하지만 등산 장비에 대해 물어...  
2                    저희 텐트 종류는 다양해요! 🏕️\n\n1. **돔 텐트**: 가벼워서 이동하기 좋...  
3                    죄송하지만, TrekReady 하이킹 부츠의 가격은 정확히 알 수 없어요. 😅 대부...  
4                    BaseCamp 접이식 테이블은 캠핑이나 야외 활동에 적합한 휴대용 테이블이에요. ...  
5                    저희의 반품 및 교환 정책은 구매 후 30일 이내에 가능합니다. 물품은 사용하지 않...  
6                    물론이죠! 사용한 텐트라도 문제가 있다면 반품을 도와드릴 수 있습니다. 구매하신 곳...